In [ ]:
from telegram import (
    Poll,
    ForceReply,
    ParseMode,
    KeyboardButton,
    KeyboardButtonPollType,
    ReplyKeyboardMarkup,
    ReplyKeyboardRemove,
    Update,
)

from telegram.ext import (
    Updater,
    CommandHandler,
    PollAnswerHandler,
    PollHandler,
    MessageHandler,
    Filters,
    CallbackContext,
)


def start(update: Update, _: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\!',
        reply_markup=ForceReply(selective=True),
    )


def help_command(update: Update, _: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def echo(update: Update, _: CallbackContext) -> None:
    """Echo the user message."""
    print(update.message.text)
    update.message.reply_text(update.message.text)
    
    
def poll(update: Update, context: CallbackContext) -> None:
    """Sends a predefined poll"""
    x, y = update.message.text.split(' ')[1:3]
    
    questions = ["Женя🐍", 
                 "Саня🦆", 
#                  "Саня🐼", 
                 "Катя🕷"]
    if len(update.message.text.split(' ')) > 3:
        questions.extend(update.message.text.split(' ')[2:])
    message = context.bot.send_poll(
        update.effective_chat.id,
        f"Отдать {int(x)/len(questions):,} рублей с каждого {y} Всего {int(x):,} рублей",
        questions,
        is_anonymous=False,
        allows_multiple_answers=True,
    )
    # Save some info about the poll the bot_data for later use in receive_poll_answer
    payload = {
        message.poll.id: {
            "questions": questions,
            "message_id": message.message_id,
            "chat_id": update.effective_chat.id,
            "answers": 0,
        }
    }
    context.bot_data.update(payload)
    
    
def receive_poll_answer(update: Update, context: CallbackContext) -> None:
    """Summarize a users poll vote"""
    answer = update.poll_answer
    poll_id = answer.poll_id
    try:
        questions = context.bot_data[poll_id]["questions"]
    # this means this poll answer update is from an old poll, we can't do our answering then
    except KeyError:
        return
    selected_options = answer.option_ids
    answer_string = ""
    for question_id in selected_options:
        if question_id != selected_options[-1]:
            answer_string += questions[question_id] + " и "
        else:
            answer_string += questions[question_id]
    context.bot.send_message(
        context.bot_data[poll_id]["chat_id"],
        f"{update.effective_user.mention_html()} отдал за {answer_string}!",
        parse_mode=ParseMode.HTML,
    )
    context.bot_data[poll_id]["answers"] += 1
    # Close poll after three participants voted
    if context.bot_data[poll_id]["answers"] == 3:
        context.bot.stop_poll(
            context.bot_data[poll_id]["chat_id"], context.bot_data[poll_id]["message_id"]
        )


def main() -> None:
    # Create the Updater and pass it your bot's token.
    updater = Updater("1849440751:AAHRNGKsxXUzZeHFMXE7sdERRdp2OxleZtE")
    dispatcher = updater.dispatcher
    dispatcher.add_handler(CommandHandler('start', start))
    dispatcher.add_handler(CommandHandler('split_money', poll))
    dispatcher.add_handler(PollAnswerHandler(receive_poll_answer))
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))
#     dispatcher.add_handler(PollHandler(receive_quiz_answer))
#     dispatcher.add_handler(CommandHandler('preview', preview))
#     dispatcher.add_handler(MessageHandler(Filters.poll, receive_poll))
#     dispatcher.add_handler(CommandHandler('help', help_handler))

    # Start the Bot
    updater.start_polling()

    # Run the bot until the user presses Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT
    updater.idle()


if __name__ == '__main__':
    main()

In [6]:
# Read the image
img = cv2.imread('receipt.jpeg',0)
# Simple thresholding
ret, thresh1 = cv2.threshold(img,210,255,cv2.THRESH_BINARY)

cv2.imshow('gray', thresh1 )
cv2.waitKey(0) 

107

In [1]:
import pytesseract
from pytesseract import Output

img = cv2.imread('receipt.jpeg')
ret, thresh1 = cv2.threshold(img,210,255,cv2.THRESH_BINARY)
d = pytesseract.image_to_string(thresh1, lang='rus')



NameError: name 'cv2' is not defined

In [21]:
print(d)

— 16/05/2021

13:44

ж Счет № 283351 ж

Стол Столё-8

Официант: Жиганова Мария
Официант: Жиганова Мария

Бефстроганов

Домашний салат
Картофель Пай

Ледяная Жемчужина 0,5л
Мохито б/а

Моцарелла с помидорами
Пельмени домашние
Сладкий цветок

Слелые ягоды

Сырные палочки

Сэндвич с лососем

Подытог:

В том числе Бар
В том числе Кухня

Итого:

1 420.00 420.00
1 240.00 240.00
1 380.00 380.00
1 120.00 120.00
1 200.00 200.00
1 480.00 480.00
1 240.00 240.00
1 220.00 220.00
1 220.00 220.00
1 280.00 280.00
1 320.00 320.00

3120.00

760.00
2360.00

3120.00

Вознаграждение Официанту приветствуется,
но всегда остается на ваше усмотрение



In [4]:
# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)
 
#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
#erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

#skew correction
def deskew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated


In [9]:
import re
import pytesseract
from pytesseract import Output

img = cv2.imread('receipt_collection_2.jpg')
# ret, thresh1 = cv2.threshold(img,210,255,cv2.THRESH_BINARY)
d = pytesseract.image_to_string(remove_noise(img), lang='rus')

d_without_n = [t for t in d.split('\n') if len(t) != 0] 
d_without_n

if [t.split('=') for t in d_without_n] != d_without_n:
    d_without_n = [t.split('=') for t in d_without_n]
    d_without_n = [j for t in d_without_n for j in t]
d_without_n

dish = [t for t in d_without_n if (not re.match(r'\d+\.\d+', t)) and (len(t)>7)]
cost = [t for t in d_without_n if re.match(r'\d+\.\d+', t)]
cost, dish
# pd.DataFrame(dish, cost)

(['120.00 ',
  '61.00 ',
  '68.00 ',
  '47.00 ',
  '120.00',
  '61.00',
  '68.00',
  '47.00'],
 ['010101540,',
  '010101020.',
  'Картофель с маслом',
  'Нап. крабовое мясо',
  'Нап.ветчина с пряной ад',
  'Чесночный хлеб'])

# learning cv2

In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

### reading images 

In [69]:
img = cv2.imread('dolphins.jpg', cv2.IMREAD_GRAYSCALE)
cv2.imshow('dolphin', img)
cv2.waitKey(0)

102

### discovering shape 

In [70]:
img2 = cv2.imread('dolphins.jpg')
img2.shape

(417, 640, 3)

In [71]:
print("Высота: "+str(img2.shape[0]))
print("Ширина: " + str(img2.shape[1]))
print("Количество каналов: " + str(img2.shape[2]))

Высота: 417
Ширина: 640
Количество каналов: 3


### saving(writing) images 

In [72]:
cv2.imwrite('dolphin_gray.jpg', img)

True

### selecting some pixel of picture 

In [37]:
img2[0, 0]

array([228,  98,  68], dtype=uint8)

in cv2 rgb is in reverse order  => bgr 

In [38]:
(b, g, r) = img2[0, 0]

In [73]:
print(b, g, r)
print("Красный: {}, Зелёный: {}, Синий: {}".format(r, g, b))

228 98 68
Красный: 68, Зелёный: 98, Синий: 228


In [76]:
for i in img2:
    for j in i:
        j[0] = 0
#         break

In [77]:
cv2.imshow('test', img2)
cv2.waitKey(0)

102